In [139]:
"""requirements.txt:- pip install -r requirements.txt
pytubefix
langchain
faiss-cpu
sentence-transformers
openai
gradio
tiktoken"""
from pytubefix import YouTube
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
import gradio as gr

In [140]:
"""#!pip install -U -r requirements.txt
#!pip install -U langchain-community pytubefix langchain faiss-cpu sentence-transformers openai gradio tiktoken"""

'#!pip install -U -r requirements.txt\n#!pip install -U langchain-community pytubefix langchain faiss-cpu sentence-transformers openai gradio tiktoken'

In [141]:
qa = None

# Step 1: Download & Clean Transcript
def download_and_clean_transcript(url, output_path="transcript.txt"):
    yt = YouTube(url)
    caption = yt.captions.get_by_language_code('en') or yt.captions.get_by_language_code('a.en')

    if caption:
        srt_captions = caption.generate_srt_captions()
        srt_lines = srt_captions.splitlines()

        clean_lines = []
        for line in srt_lines:
            line = line.strip()
            if line.isdigit() or '-->' in line or line == '':
                continue
            clean_lines.append(line)

        with open(output_path, 'w', encoding='utf-8') as file:
            for line in clean_lines:
                file.write(line + '\n')

        return "Transcript cleaned and saved successfully!"
    else:
        return "English or auto-generated English captions not found."


In [142]:
def chat_with_doc_setup():
    path = "transcript.txt"
    loader = TextLoader(path, encoding="utf-8")
    documents = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.split_documents(documents)

    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(chunks, embedding_model)

    retriever = vectorstore.as_retriever()
    llm = ChatOpenAI(
        model="llama3-8b-8192",
        openai_api_key="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",
        openai_api_base="https://api.groq.com/openai/v1",
        temperature=0.4,
    )
    return RetrievalQA.from_chain_type(llm=llm, retriever=retriever)


In [143]:
def summarize_video(url):
    status = download_and_clean_transcript(url)
    if "successfully" not in status:
        return status

    global qa
    qa = chat_with_doc_setup()
    return qa.run("Summarize this YouTube video like a course breakdown. Include the key topics covered, practical tips, learning advice. Be clear, structured, and helpful for a beginner.")

In [144]:
def ask_question_about_video(question, history):
    global qa
    if qa is None:
        return [("System", "Please summarize a video first.")], ""
    answer = qa.run(question)
    history = history or []
    history.append((question, answer))
    return history, ""

In [145]:
custom_logo_html = """
<div style="display: flex; align-items: center; gap: 10px; padding: 10px; background-color: #181818; color: white;">
    <svg height="30" viewBox="0 0 24 24" width="30" fill="none" xmlns="http://www.w3.org/2000/svg">
        <rect x="1.5" y="1.5" width="21" height="21" rx="4" stroke="#FFA500" stroke-width="2"/>
        <polygon points="10,8 16,12 10,16" fill="#800080"/>
    </svg>
    <h1 style="font-size: 24px; font-weight: 600; margin: 0;">VideoBrain</h1>
</div>
"""
from huggingface_hub import create_repo
with gr.Blocks() as demo:
    gr.HTML(custom_logo_html)
    gr.Markdown("## Created by Krishna Bhat U")
    youtube_url = gr.Textbox(label="Enter YouTube Video Link")
    summarize_button = gr.Button("Summarize")
    summary_output = gr.Textbox(label="📝 Video Summary", lines=20)

    gr.Markdown("## 💬 Q&A Chat")

    chat_history = gr.Chatbot()
    question_input = gr.Textbox(label="Ask a question about the video")
    ask_button = gr.Button("Send")


    summarize_button.click(fn=summarize_video, inputs=youtube_url, outputs=summary_output)
    ask_button.click(fn=ask_question_about_video, inputs=[question_input, chat_history], outputs=[chat_history, question_input])
demo.launch(share=True)

<ipython-input-145-8e0dd07652d3>:20: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chat_history = gr.Chatbot()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dec0685edbb5c7c98e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
